In [ ]:
'''目标：
1，爬取豆瓣电影top250
2，爬取电影详情信息
3，爬取演员详情信息
'''

In [1]:
'''接口：
1,  TOP 250 	http://api.douban.com/v2/movie/top250 	start: 数据的开始项;count: 单页条数

'''

'接口：\n\n'

In [81]:
import json
import requests
import os
from tqdm import tqdm_notebook
import operator

In [73]:
#新建文件夹
if not os.path.exists('top250'):
    os.mkdir('top250')

In [124]:
#爬取全部250部电影
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
for i in tqdm_notebook(range(25)):
    url = f'https://api.douban.com/v2/movie/top250?apikey={access_key}&start={i*10}&count={10}'
    
    json_file_path = os.path.join('top250', '{}.json'.format(i))
    if os.path.exists(json_file_path):  #若已爬过，跳过
        continue  
        
    #防止超时，访问接口直到访问成功
    while True:
        try:
            response = requests.get(url,headers = headers,timeout = 5)
            break
        except requests.exceptions.Timeout:
            time.sleep(1)
            
    #写入文件
    with open(json_file_path,'w',encoding='utf-8') as f:
        f.write(json.dumps(json.loads(response.content.decode('utf-8')),indent=4,ensure_ascii=False))
        #indent=4:对齐 ensure_ascii=False:编码中文 json.dumps()用于将字典形式的数据转化为字符串，json.loads()用于将字符串形式的数据转化为字典

In [125]:
#将上面取的文件合并为top250.json
movies = []
for json_file in tqdm_notebook(os.listdir('top250')):
    json_file_path = os.path.join('top250', json_file)
    with open(json_file_path,'r',encoding = 'utf8') as f:
        
        movies += json.load(f)['subjects']

#按平均分降序排序
movies = sorted(movies,
               key = lambda movie: movie['rating']['average'],reverse= True)

#写入文件
with open('top250.json','w',encoding = 'utf8') as f:
    f.write(json.dumps({'subjects':movies},indent=4,ensure_ascii=False))
    